# Black Scholes Model


We start by importing modules that are needed to run this file

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt # Plotting tools
from scipy.stats import norm # For Black Scholes Model
from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

import warnings
warnings.simplefilter('ignore')

We then define functions to evaluate option prices

Terminology
S: underlying stock price
K: Option strike price
r: risk free rate
D: dividend value
vol: Volatility
T: time to expiry (assumed that we're measuring from t=0 to T)

In [2]:
## Functions to calculate BSM
def d1_calc(S, K, r, vol, T, t):
    # Calculates d1 in the BSM equation
    return (np.log(S/K) + (r + 0.5 * vol**2)*(T-t))/(vol*np.sqrt(T-t))

def BS_call(S, K, r, vol, T, t):
    d1 = d1_calc(S, K, r, vol, T, t)
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2)

def BS_put(S, K, r, vol, T, t):
    return BS_call(S, K, r, vol, T, t) - S + np.exp(-r*(T-t))*K

def option_price(K,r,vol,T,t,option):
    S = [t/5 for t in range(0,200)]
    if option == 'call':
        cval = [BS_call(x, K, r, vol, T, t) for x in S]
        plt.plot(S, cval, 'r', label = "Call")
        plt.legend()
        plt.ylabel("Stock Price ($)")
        plt.xlabel("Option Price ($)")
        plt.show()
    elif option == 'put':
        pval = [BS_put(x, K, r, vol, T, t) for x in S]
        plt.plot(S, pval, 'b', label  = "Put")
        plt.legend()
        plt.ylabel("Stock Price ($)")
        plt.xlabel("Option Price ($)")
        plt.show()
    else:
        cval = [BS_call(x, K, r, vol, T, t) for x in S]
        pval = [BS_put(x, K, r, vol, T, t) for x in S]
        plt.plot(S, cval, 'r', label = "Call")
        plt.plot(S, pval, 'b', label  = "Put")
        plt.legend()
        plt.ylabel("Stock Price ($)")
        plt.xlabel("Option Price ($)")
        plt.show()

We use widgets to interact with the function. Here, we can choose values of K,r,vol,T,t and the option type

In [3]:
# Stock price in this case is defined by S. It goes from 0 to about 40
interact(option_price, K=(1,40,1), r=(0,1.0,0.1), vol=(0,0.5,0.1), T=(0,10,1), t=(0,10),
         option={'call','put','both'});

# Recommended values to start
# K = 10
# r = 0.10
# vol = 0.2
# T = 10
# t = 0

interactive(children=(IntSlider(value=20, description='K', max=40, min=1), FloatSlider(value=0.5, description=…

We define functions to evaluate Greeks

In [4]:
## Functions to calculate Greeks

#1st Order Greeks
def delta(S, K, r, vol, T, t, otype):
    d1 = d1_calc(S, K, r, vol, T, t)
    d2 = d1 - vol * np.sqrt(T-t)
    
    if(otype == "call"):
        delta = np.exp(-(T-t))*norm.cdf(d1)
    elif(otype == "put"):
        delta = -np.exp(-(T-t))*norm.cdf(-d1)

    return delta

# Gamma for calls/puts the same

def vega(S, K, r, vol, T, t, otype):
    d1 = d1_calc(S, K, r, vol, T, t)
    return S * norm.pdf(d1) * np.sqrt(T-t)

def rho(S, K, r, vol, T, t, otype):
    d1 = d1_calc(S, K, r, vol, T, t)
    d2 = d1 - vol*np.sqrt(T-t)
    
    if(otype == "call"):
        rho = K*(T-t)*np.exp(-r*(T-t))*norm.cdf(d2)
    elif(otype == "put"):
        rho = -K*(T-t)*np.exp(-r*(T-t))*norm.cdf(-d2)
    return rho

def theta(S, K, r, vol, T, t, otype):
    d1 = d1_calc(S, K, r, vol, T, t)
    d2 = d1 - vol*np.sqrt(T-t)
    
    if(otype == "call"):
        theta = -(S*norm.pdf(d1)*vol / (2*np.sqrt(T-t))) - r*K*np.exp(-r*(T-t))*norm.cdf(d2) 
    elif(otype == "put"):
        theta = -(S*norm.pdf(d1)*vol / (2*np.sqrt(T-t))) + r*K*np.exp(-r*(T-t))*norm.cdf(-d2)

    return theta

#2nd Order Greeks
def gamma(S, K, r, vol, T, t, otype):
    d1 = d1_calc(S, K, r, vol, T, t)
    gamma = (norm.pdf(d1)) / (S * vol * np.sqrt(T-t))
    
    return gamma

def charm(S, K, r, vol, T, t, otype):
    d1 = d1_calc(S, K, r, vol, T, t)
    d2 = d1 - vol*np.sqrt(T-t)
    charm = -norm.pdf(d1)*(2*r*(T-t) - d2*vol*np.sqrt(T-t))/(2*(T-t)*vol*np.sqrt(T-t))

    return charm

def greeks(K,r,vol,T,t, otype, greekfun):
    S = [t/5 for t in range(0,200)]
    if otype=='call':
        color = 'r'
    else:
        color = 'b'
    val = [greekfun(x, K, r, vol, T, t, otype)for x in S]
    plt.plot(S, val, color, label = otype)
    plt.legend()
    plt.ylabel(greekfun.__name__)
    plt.xlabel("Underlying Price")
    plt.show()


Again, we use widgets to interact with the functions.

In [5]:
interact(greeks, K=(1,40,1), r=(0,1.0,0.1), vol=(0,0.5,0.1), T=(0,10,1), t=(0,10),
         otype={'call','put'},
         greekfun={'delta':delta,'vega':vega,'rho':rho,'theta':theta,'gamma':gamma,'charm':charm});

# Recommended values to start
# K = 10
# r = 0.10
# vol = 0.2
# T = 10
# t = 0

interactive(children=(IntSlider(value=20, description='K', max=40, min=1), FloatSlider(value=0.5, description=…